# =========================================================
# PolyEpi: Polygenic Phenotype with Higher-Order Epistasis Interactios
# =========================================================

In [ ]:
#! pip3 install pdbio

# Path to the config file (Modify this before running the notebook.

In [ ]:
configFilePath='SampleData/config-peps3.json'
PEPS3=True # if flase then PEPS2

# Initialisation

In [ ]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json
from collections import Counter
from random import random

from pprint import pprint
from pdbio.vcfdataframe import VcfDataFrame

import itertools

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from scipy import stats
import sklearn

TA = [1, .5, 1e-1, 1e-2, 1e-3, 1e-8, 1e-20]

# Read the config file

In [ ]:
with open(configFilePath, 'r') as f:
    config = json.load(f)

In [ ]:
shuffleSnps = config['shuffleSnps']
seed = config['seed']

np.random.seed(None if shuffleSnps else seed)

inputType = config['inputType']
dumpCSV = config['dumpCSV']

vcfInputPath = config['inputPrefix']+'.vcf'
csvInputPath = config['inputPrefix']+'.csv'

outputPrefix = config['outputPrefix']

numTree = config['numTree']

# Compute total numebr of variables and number of requierd SNPs to form the variables

In [ ]:
numVariables = 0
numSnpsNeeded = 0
maxOrder = 0

for v in config['variables']:
    v['numSNPs'] = v['numVar'] * v['numSnpsInVar']
    numVariables += v['numVar']
    numSnpsNeeded += v['numSNPs']
    if v['numSnpsInVar']>maxOrder:
        maxOrder = v['numSnpsInVar']
    
config['numVariables'] = numVariables
config['numSnpsNeeded'] = numSnpsNeeded
config['maxOrder'] = maxOrder

# Print config data and write it in "outputPrefix.config.json"

In [ ]:
! ls PEPS/SampleData/

In [ ]:
print("========== Configuration >>>")
pprint(config)
print("============================")
with open(outputPrefix+'.config.json','w+') as outfile:
    json.dump(config, outfile, indent=4, sort_keys=True)

# Parse input genotype data from a VCF or CSV file
## If read from VCF file, the SNP id is set to CHROM:POS:REF:ALT

In [ ]:
if inputType=='vcf':
    vcfdf = VcfDataFrame(path=vcfInputPath)
    df = vcfdf.df
    df['SNP'] = df['#CHROM'].astype(str) + ':' + df['POS'].astype(str) + ':' + df['REF'].astype(str) + ':' + df['ALT'].astype(str)
    df = df.set_index('SNP')
    snpData = df.iloc[:,9:].replace(['0/0','0/1','1/1'], [0,1,2])
    if dumpCSV:
        snpData.to_csv(csvInputPath)
elif inputType=='csv':
    snpData = pd.read_csv(csvInputPath)
    snpData = snpData.set_index('SNP')
else:
    print("Incorrect inputType (should be 'vcf' or 'csv')")
    exit()

In [ ]:
snpData.iloc[:5,:5]

# There should be enough SNPs in the input file to create all variables

In [ ]:
print("Number of SNPs in the input file: ", snpData.shape[0])
print("Number of SNPs needed: ", numSnpsNeeded)

if snpData.shape[0] < numSnpsNeeded:
    print("There are not enough SNPs in the input file")
    exit(1)
else:
    print("There are enough SNPs in the input file")

# Suffle SNPs if asked in the config file.
## When SNPs are shuffled different set of SNPs used to form each variables each time

In [ ]:
for col in snpData.columns.tolist():
    snpData[col] = snpData[col].sample(frac=1).tolist()
snpData

# Transpose the genotype data and print number of snps and samples
## Also rename 0/0, 0/1 and 1/1 to R, H and A

In [ ]:
snpData = snpData.T

In [ ]:
df = snpData.replace([0,1,2],['R','H','A'])
numSamples = df.shape[0]
numSNPs = df.shape[1]
print("number of sample",numSamples)
print("number of snp",numSNPs)

# Form variables from SNPs
## A variable could be a SNPs or a set of Interactive SNPs
## First identify which SNPs belong to each variable and then form the variables
## Naming of variables: O3V4 is the 4th variable with 3-interactive SNPs. Then each of these SNPs will form a Variable (O3V5S1, O3V5S2, O3V5S3)
## Write Variables SNPs infor in "outputPrefix.varData.csv"

In [ ]:
colNames = list() # to store variable names
for o,v in enumerate(config['variables']):
    for i in range(0,v['numVar']):
        colNames.append('O'+str(o+1)+'V'+str(i+1))
        # PEPS3 (Start)
        if PEPS3:
            if o > 0:
                for s in range(0,o+1):
                    colNames.append('O'+str(o+1)+'V'+str(i+1)+'S'+str(s+1))
        # PEPS3 (END)

In [ ]:
rowNames = ['order']
for o in range(maxOrder):
    rowNames.append('snp_'+str(o+1))

In [ ]:
varData = pd.DataFrame(index=rowNames, columns=colNames)

In [ ]:
idx = 0
for o,v in enumerate(config['variables']):
    for i in range(0,v['numVar']):
        name = 'O'+str(o+1)+'V'+str(i+1)
        varData.at['order',name] = str(o+1)
        for k in range(0,v['numSnpsInVar']):
            snp = 'snp_'+str(k+1)
            varData.at[snp,name]=df.columns[idx]
            # PEPS3 (Start)
            if PEPS3:
                if o > 0:
                    name3 = 'O'+str(o+1)+'V'+str(i+1)+'S'+str(k+1)
                    varData.at['order',name3] = 1
                    varData.at['snp_1',name3] = df.columns[idx]
            # PEPS3 (END)
            idx += 1
varData = varData.fillna('---')

In [ ]:
varData

In [ ]:
varData.to_csv(outputPrefix+'.varData.csv')

In [ ]:
varData.iloc[:6,-10:]

# Form Variable Genotype and write it to "outputPrefix.varGT.csv"
## For variables with more than one SNPs the genotype is the concatination of all SNPs involved
## For example RHA, ARH and AAR could be genotype value of a variable with 3 snps

In [ ]:
varGT = df.iloc[:,-1:0].copy()

In [ ]:
# PEPS3 block
if PEPS3:
    for name in varData.columns:
        o = int(varData.loc['order',name])
        print(name,o)
        varGT[name] = ''
        for k in range(0,o):
            snp = 'snp_'+str(k+1)
            varGT[name] = varGT[name] + df[varData.loc[snp,name]]

In [ ]:
if ~PEPS3:
    for o,v in enumerate(config['variables']):
        for i in range(0,v['numVar']):
            name = 'O'+str(o+1)+'V'+str(i+1)
            varGT[name] = ''
            for k in range(0,v['numSnpsInVar']):
                snp = 'snp_'+str(k+1)
                varGT[name] = varGT[name] + df[varData.loc[snp,name]]

In [ ]:
varGT

In [ ]:
varGT[[x for x in varGT.columns if 'O5V2' in x]]

In [ ]:
varGT.to_csv(outputPrefix+'.varGT.csv')

In [ ]:
varGT.iloc[:5,-5:]

In [ ]:
PEPS2_Input = varGT.T.values.tolist()

# PEPS2 Simulation

In [ ]:
def assign(genotypes, case_freq):
    """Create a list representing Cases (1) and Controls (0)
​
    Assigns cases and controls to the samples (in order) such that all
    variables have a similar impact on the phenotype, and the case frequency is
    case_freq.
​
    :param genotypes: A list of lists, where the primary index is variable, and
        the secondary index is sample. Elements represent the value taken by
        that variable in that sample. the most common value doesn't contribute
        to the phenotype.
    :param case_freq: The desired frequency of the case in the population.
    :return: a list of the phenotypes of the samples
    """
    num_samples = len(genotypes[0])
    global_base_case = num_samples * (
        1 - (1-case_freq) ** (1 / len(genotypes))
    )
    variable_likelihoods = []
    for variable_values in genotypes:
        value_counts = Counter(variable_values)
        del value_counts[value_counts.most_common(1)[0][0]]  # remove most common genotype
        num_values = len(value_counts)
        variable_likelihoods.append({
            k: 1 - min(global_base_case / (num_values * v), 1)
            for k, v in value_counts.items()
        })
    sample_likelihoods = []
    for s_i in range(num_samples):
        control_prob = 1
        for v_i, variable in enumerate(genotypes):
            control_prob *= variable_likelihoods[v_i].get(variable[s_i], 1)
        sample_likelihoods.append((control_prob, s_i))
    sample_phenotypes = [0] * num_samples
    sample_likelihoods.sort()
    for _, s_i in sample_likelihoods[:int(case_freq * num_samples)]:
        sample_phenotypes[s_i] = 1
    return sample_phenotypes

In [ ]:
PEPS2_Output = assign(PEPS2_Input,0.5)

In [ ]:
PEPS2_Output[:10]

In [ ]:
varGT['lbl'] = PEPS2_Output

# Compute and plot chi2-pvalue (log10) of the variables for the random phenotype

In [ ]:
features = varGT.columns[:-1]
corrDict = dict()
for v in features:
    corrDict[v] = stats.chi2_contingency(pd.crosstab(varGT['lbl'],varGT[v]).values)[1]
a = np.asarray(list(corrDict.values()))
b = - np.log10(a)
plt.plot(np.sort(b))
nsat = list()
for t in TA:
    nsat.append([t, np.where(a<t)[0].shape[0]])
x = pd.DataFrame(nsat)
x.columns =['p-value', 'number of vars exceed the p-value']
x.set_index('p-value')

In [ ]:
corrDict

# Write Phenotype into a file outputPrefix.pheno.csv

In [ ]:
phen = varGT[['lbl']].copy()
phen.index.name ='sample'
phen.to_csv(outputPrefix+'.pheno.csv')

In [ ]:
phen.head()

In [ ]:
varData.to_csv(outputPrefix+'.varDataTruth.csv')

In [ ]:
varData.iloc[:6,-5:]

# Filter SNPs included in All and Truth Variables
## write the Truth SNP names in  "outputPrefix.TruthSNP.csv"

In [ ]:
snps = np.unique(varData.replace(np.nan, '', regex=True).drop('order').values.ravel())[1:]
snpDataVar = snpData.loc[:, snpData.columns.isin(snps)].copy()

pd.DataFrame(snps).rename(columns={0:'v'}).to_csv(outputPrefix+'.TruthSNP.csv',index=False)

In [ ]:
snpData

In [ ]:
num_truth_snp = snpDataVar.shape[1]
print("Number of SNPs used to form Variables: ",num_truth_snp)

In [ ]:
snpDataVar.iloc[:5,-5:]

# Add Phenotype to SNP Data

In [ ]:
snpDataVar.loc[:, 'lbl'] = varGT['lbl']
snpData.loc[:, 'lbl'] = varGT['lbl']

# Compute and plot chi2-pvalue (log10) of the variables for the random phenotype

In [ ]:
features = snpDataVar.columns[:-1]
corrDict = dict()
for v in features:
    corrDict[v] = stats.chi2_contingency(pd.crosstab(snpDataVar['lbl'],snpDataVar[v]).values)[1]
a = np.asarray(list(corrDict.values()))
b = - np.log10(a)
plt.plot(np.sort(b))
nsat = list()
for t in TA:
    nsat.append([t, np.where(a<t)[0].shape[0]])
x = pd.DataFrame(nsat)
x.columns =['p-value', 'number of SNPs exceed the p-value']
x.set_index('p-value')

# This function used to predict lable using RandomForest
## 75% training and 25% test

In [ ]:
def RF_AUC(dfx, nTree):
    df = dfx.copy()
    features = df.columns[:-1]
    
    df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
    train, test = df[df['is_train']==True], df[df['is_train']==False]
    clf = RandomForestClassifier(n_jobs=2, n_estimators=nTree, random_state=0)
    clf.fit(train[features], train['lbl'])
    prob = clf.predict_proba(test[features])
    y_true = test['lbl']
    y_scores = prob[:,1]
    return clf, roc_auc_score(y_true, y_scores)


# Train and test RandomForest for TruthSNP as well as for all SNP in the input file
## Print AUC
## Plot Importance Score

In [ ]:
aucvars = []

In [ ]:
clfVar,   AucVar   = RF_AUC(snpDataVar  , nTree=numTree)
aucvars.append(round(AucVar, 3))
print("AUC All SNPs in Variables : ", aucvars)

In [ ]:
clfVar,   AucVar   = RF_AUC(snpDataVar  , nTree=numTree)
aucvars.append(round(AucVar, 3))
print("AUC All SNPs in Variables : ", aucvars)

In [ ]:
clfVar,   AucVar   = RF_AUC(snpDataVar  , nTree=numTree)
aucvars.append(round(AucVar, 3))
print("AUC All SNPs in Variables : ", aucvars)

In [ ]:
clfVar,   AucVar   = RF_AUC(snpDataVar  , nTree=numTree)
aucvars.append(round(AucVar, 3))
print("AUC All SNPs in Variables : ", aucvars)

In [ ]:
clfVar,   AucVar   = RF_AUC(snpDataVar  , nTree=numTree)
aucvars.append(round(AucVar, 3))
print("AUC All SNPs in Variables : ", aucvars)
print(round(sum(aucvars)/len(aucvars), 3))

In [ ]:
clf,      Auc      = RF_AUC(snpData     , nTree=numTree)
importance_indexes =  [(snp_import, snp_index) for snp_index, snp_import in enumerate(clf.feature_importances_)]
importance_indexes.sort(reverse=True)

In [ ]:
num_top = num_truth_snp * 2
top_snp_indexes = [snp[1] for snp in importance_indexes[:num_top]]
topSnps = snpData.iloc[:, top_snp_indexes + [-1]]
clfTop,      AucTop      = RF_AUC(topSnps, nTree=numTree)
print("AUC All SNPs in Variables : ", round(AucVar, 3))
print("AUC All SNPs in input file: ", round(Auc, 3))
print("Top " + str(num_top) + " SNPs: ", round(AucTop, 3))
pd.DataFrame(np.sort(clfVar.feature_importances_)).plot(title="All SNP in Variables")
pd.DataFrame(np.sort(clf.feature_importances_)).plot(title="All SNP in input file")
pd.DataFrame(np.sort(clfTop.feature_importances_)).plot(title="Top " + str(num_top) + " SNPs in input file")

In [ ]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(snpData.columns, clf.feature_importances_):
    feats[feature] = importance #add the name/value pair 
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})

In [ ]:
ix = importances.sort_values('Gini-importance', ascending=False)

In [ ]:
ix['rank'] = range(1,ix.shape[0]+1)

In [ ]:
ix.shape

In [ ]:
Truth = pd.DataFrame(snps).rename(columns={0:'v'}).set_index('v')

In [ ]:
Truth.shape

In [ ]:
jx = ix.join(Truth,how='inner')

# Rank and importance of truth snp when processing all SNP in the file

In [ ]:
from IPython.core.display import HTML
display(HTML(jx.reset_index().to_html()))

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
os.system("jupyter nbconvert PEPS3.ipynb --to html --output "+outputPrefix+".html")

In [ ]:
outputPrefix